In [ ]:
import pandas as pd

import import_ipynb
from preprocessor import preprocessor, drop_index, read_file
from tree import Tree

result_path = '../result_kor_OSX.csv'
result_out_path = 'result_out.csv'

def predict(condition, model):
    target_columns = condition.target_columns
    for target_column in target_columns:
        dropping_columns = target_columns.drop(target_column)
        X, y = preprocessor.preprocessed(target_column, dropping_columns)
        b = X.columns
        model.train(X, y, target_column, dropping_columns)
    predicted = {}
    for target_column in target_columns:
        dropping_columns = target_columns.drop(target_column)
        a = condition.preprocessed.index
        preprocessed = condition.preprocessed.values.reshape(1, -1)
        predicted[target_column] = model.predict(
            preprocessed, target_column, dropping_columns)[0]
    return pd.Series(predicted)

def char_to_int(char):
    return ord(char.lower()) - 96

model = Tree()
completed_rows = []
for condition in preprocessor.conditions:
    original = drop_index(condition.original.dropna(), condition.target_columns)
    predicted = preprocessor.restore_predicted(predict(condition, model))
    completed_row = pd.concat([original, predicted])
    completed_rows.append(completed_row)
completed = (pd.concat(completed_rows, axis=1, sort=False).T
               .reindex(columns=preprocessor.predict_columns_order))
completed['사상자수'] = (completed.reindex(columns=['사망자수', '중상자수', '경상자수'])
                                .apply(sum, axis=1))

result = read_file(result_path)
rows = []
for index, row in result.iterrows():
    answer = completed.loc[row['열'] - 2][char_to_int(row['행']) - 1]
    row['값'] = answer
    rows.append(row)
result_out = pd.DataFrame(rows)
# result_out.to_csv(result_out_path)

Training target:  사망자수  (dropped Index(['경상자수'], dtype='object'))
    Test score: 0.9069591134069185, (base score: 0.9668091268418382)
Training target:  경상자수  (dropped Index(['사망자수'], dtype='object'))
    Test score: 0.9110051245751773, (base score: 0.9113787339852367)
Training target:  사망자수  (dropped Index(['경상자수'], dtype='object'))
Training target:  경상자수  (dropped Index(['사망자수'], dtype='object'))
Training target:  중상자수  (dropped Index(['부상신고자수'], dtype='object'))
    Test score: 0.8765544953491187, (base score: 0.8762999464342794)
Training target:  부상신고자수  (dropped Index(['중상자수'], dtype='object'))
    Test score: 0.9795684491793273, (base score: 0.9828530191957248)
Training target:  중상자수  (dropped Index(['부상신고자수'], dtype='object'))
Training target:  부상신고자수  (dropped Index(['중상자수'], dtype='object'))
Training target:  중상자수  (dropped Index(['경상자수'], dtype='object'))
